# Hierarchically Deep Convolutional Neural Network For Image Recognition

## Setup and Imports

**Import Packages**

In [ ]:
import keras as kr
import numpy as np
import tensorflow as tf

from keras.datasets import cifar100

from sklearn.model_selection import train_test_split

from random import randint
import time
import os
import shutil
import zipfile
import matplotlib.pyplot as plt
import csv
from PIL import Image
from sklearn.preprocessing import LabelEncoder

**Define Global Variables**

In [ ]:
# The number of coarse categories
coarse_categories = 8

# The number of fine categories
fine_categories = 180

##BUILD ROBOCUP_2019 Dataset

###Dataset details
RoboCup Dataset contains 196195 images.

8 main parent categories that contain 180 children categories.

70% for train

30% for test


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ruta_archivo_zip = './drive/MyDrive/dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
def load_data(image_size):
  images = []
  labels_c = []
  labels = []

  with open('./data/train.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for item in data:
      img = Image.open(data[0][0].replace('\\','/'))
      img = img.resize((32,32))
      img = np.array(img)
      images.append(img)
      labels_c.append(item[1])
      labels.append(item[2])
    x_train = np.array(images, dtype=np.uint8)
    y_train_c = np.array(labels_c)
    y_train = np.array(labels)
    encoder = LabelEncoder()
    y_train_c = encoder.fit_transform(y_train_c) #convertir en dtype uint8
    y_train = encoder.fit_transform(y_train)
    y_train_c = y_train_c.reshape((-1,1))    #shape = (137337,) to (137337,1)
    y_train = y_train.reshape((-1,1))
    print("x_train shape:", x_train.shape)
    print("y_train_c shape:", y_train_c.shape)
    print("y_train shape:", y_train.shape)

  images.clear()
  labels_c.clear()
  labels.clear()

  with open('./data/test.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)
    for item in data:
      img = Image.open(data[0][0].replace('\\','/'))
      img = img.resize((32,32))
      img = np.array(img)
      images.append(img)
      labels_c.append(item[1])
      labels.append(item[2])
    x_test = np.array(images, dtype=np.uint8)
    y_test_c = np.array(labels_c)
    y_test = np.array(labels)
    encoder = LabelEncoder()
    y_test_c = encoder.fit_transform(y_test_c) #convertir en dtype uint8
    y_test = encoder.fit_transform(y_test)
    y_test_c = y_test_c.reshape((-1,1))    #shape = (58859,) to (58859,1)
    y_test = y_test.reshape((-1,1))
    print("x_test shape:", x_test.shape)
    print("y_test_c shape:", y_test_c.shape)
    print("y_test shape:", y_test.shape)

  return (x_train, y_train_c, y_train),(x_test, y_test_c, y_test)

**Load RoboCup Data Set**

In [ ]:
(X, y_c, y), (x_test, y_c_test, y_test) = load_data(image_size=32)

x_train shape: (137337, 32, 32, 3)
y_train_c shape: (137337, 1)
y_train shape: (137337, 1)
x_test shape: (58859, 32, 32, 3)
y_test_c shape: (58859, 1)
y_test shape: (58859, 1)


##Preprocess Dataset

**Import Cifar100 Data Set**

In [ ]:
#(X, y_c), (x_test, y_c_test) = cifar100.load_data(label_mode='coarse')
#(X, y), (x_test, y_test) = cifar100.load_data(label_mode='fine')

**Fine-To-Coarse Mapping**

(Ideally, this would be done through spectral clustering as opposed to hard-coding)

In [ ]:
fine2coarse = np.zeros((fine_categories,coarse_categories))
for i in range(coarse_categories):
    index = np.where(y_c_test[:,0] == i)[0]
    fine_cat = np.unique([y_test[j,0] for j in index])
    for j in fine_cat:
        fine2coarse[j,i] = 1


In [ ]:
y_c = 0; # Clear y_c in interest of saving mem
y_c_test=0;

In [ ]:
################################################################################
#    Title: One Hot Encoding
################################################################################
#    Description:
#        This function extends a matrix to one-hot encoding
#
#    Parameters:
#        y    Array of label values
#
#    Returns:
#        y_new    One hot encoded array of labels
################################################################################
def one_hot(y):
    n_values = np.max(y) + 1
    y_new = np.eye(n_values)[y[:,0]]
    return y_new

In [ ]:
y=one_hot(y)
y_test=one_hot(y_test)
print(np.shape(y))

(137337, 180)


**Apply ZCA Whitening**

La transformación ZCA Whitening se utiliza principalmente para reducir la correlación entre los píxeles de las imágenes y mejorar la capacidad de los algoritmos de aprendizaje para extraer características útiles de las mismas.

In [ ]:
################################################################################
#    Title: ZCA
################################################################################
#    Description:
#        This function applies ZCA Whitening to the image set
#
#    Parameters:
#        x_1           Array of MxNxC images to compute the ZCA Whitening
#        x_2           Array of MxNxC images to apply the ZCA transform
#        num_batch    Number of batches to do the computation
#
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def zca(x_1, x_2, epsilon=1e-5):

    with tf.name_scope('ZCA'):
        tf.compat.v1.disable_eager_execution()

        x1 = tf.compat.v1.placeholder(tf.float64, shape=np.shape(x_1), name='placeholder_x1')
        x2 = tf.compat.v1.placeholder(tf.float64, shape=np.shape(x_2), name='placeholder_x2')

        flatx = tf.cast(tf.reshape(x1, (-1, np.prod(x_1.shape[-3:])),name="reshape_flat"),tf.float64,name="flatx")
        sigma = tf.tensordot(tf.transpose(flatx),flatx, 1,name="sigma") / tf.cast(tf.shape(flatx)[0],tf.float64) ### N-1 or N?
        s, u, v = tf.compat.v1.svd(sigma,name="svd")
        pc = tf.tensordot(tf.tensordot(u,tf.compat.v1.diag(1. / tf.sqrt(s+epsilon)),1,name="inner_dot"),tf.transpose(u),1, name="pc")

        net1 = tf.tensordot(flatx, pc,1,name="whiten1")
        net1 = tf.reshape(net1,np.shape(x_1), name="output1")

        flatx2 = tf.cast(tf.reshape(x2, (-1, np.prod(x_2.shape[-3:])),name="reshape_flat2"),tf.float64,name="flatx2")
        net2 = tf.tensordot(flatx2, pc,1,name="whiten2")
        net2 = tf.reshape(net2,np.shape(x_2), name="output2")

    with tf.compat.v1.Session() as sess:
            sess.run(tf.compat.v1.global_variables_initializer())
            x_1,x_2 = sess.run([net1,net2], feed_dict={x1: x_1, x2: x_2})
    return x_1,x_2

In [ ]:
time1 = time.time()
X,x_test = zca(X,x_test)
time2 = time.time()
print('Time Elapsed - ZCA Whitening: '+str(time2-time1));

Time Elapsed - ZCA Whitening: 19.791391611099243


**Split Training set into Training and Validation sets**

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=.1, random_state=0)
X = 0
y = 0

**Flip, pad and randomly crop each photo**

In [ ]:
################################################################################
#    Title: Preprocess Img
################################################################################
#    Description:
#        This function pads images by 4 pixels, randomly crops them, then
#        randomly flips them
#
#    Parameters:
#        x_1           Array of MxNxC images to compute the ZCA Whitening
#        x_2           Array of MxNxC images to apply the ZCA transform
#        num_batch    Number of batches to do the computation
#
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def preprocess_img(X,y):

    with tf.name_scope('Preproc'):
        tf.compat.v1.disable_eager_execution()

        images = tf.compat.v1.placeholder(tf.float64, shape=np.shape(X))
        labels = tf.compat.v1.placeholder(tf.float64, shape=np.shape(y))

        net = tf.map_fn(lambda img: tf.image.flip_left_right(img), images)
        net = tf.map_fn(lambda img: tf.image.rot90(img), net)
        net = tf.compat.v1.image.resize_image_with_crop_or_pad(net,40,40)
        net = tf.map_fn(lambda img: tf.compat.v1.random_crop(img, [32,32,3]), net)

        net1 = tf.compat.v1.image.resize_image_with_crop_or_pad(images,40,40)
        net1 = tf.map_fn(lambda img: tf.compat.v1.random_crop(img, [32,32,3]), net1)

        net = tf.concat([net, net1],0)
        net = tf.compat.v1.random_shuffle(net, seed=0)
        net_labels = tf.concat([labels, labels],0)
        net_labels = tf.compat.v1.random_shuffle(net_labels,seed=0)

        net = tf.map_fn(lambda img: tf.image.random_flip_up_down(img), net)

    with tf.compat.v1.Session() as sess:
            sess.run(tf.compat.v1.global_variables_initializer())
            x_t,y_t = sess.run([net,net_labels], feed_dict={images: X, labels: y})
    return x_t,y_t


## Single Classifier Training

**Constructing CNN**

In [ ]:
from keras import optimizers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from keras.models import Model

in_layer = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

net = Conv2D(384, 3, strides=1, padding='same', activation='elu')(in_layer)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(384, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(384, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(640, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(640, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.2)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(640, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.3)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(768, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(896, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(896, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.4)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(896, 3, strides=1, padding='same', activation='elu')(net)
net = Conv2D(1024, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(1024, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.5)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(1152, activation='elu')(net)
#net = Dense(100, activation='softmax')(net)
net = Dense(180, activation='softmax')(net)

**Compile Model**

In [ ]:
model = Model(inputs=in_layer,outputs=net)
sgd_coarse = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer= sgd_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#model.load_weights('data/models/model_coarse'+str(30))

**Train Model**

In [ ]:
tbCallBack = kr.callbacks.TensorBoard(log_dir='./data/graph/elu_drop/', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
batch = 64

In [ ]:
index= 0
step = 5
stop = 30

while index < stop:
    model.fit(x_train, y_train, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val), callbacks=[tbCallBack])
    index += step
    model.save_weights('data/models/model_coarse'+str(index))
save_index = index

Train on 123603 samples, validate on 13734 samples
Epoch 1/5
123603/123603 [==============================] - ETA: 0s - loss: 5.1889 - accuracy: 0.0065

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


123603/123603 [==============================] - 181s 1ms/sample - loss: 5.1889 - accuracy: 0.0065 - val_loss: 5.1862 - val_accuracy: 0.0068
Epoch 2/5
123603/123603 [==============================] - 172s 1ms/sample - loss: 5.1876 - accuracy: 0.0069 - val_loss: 5.1872 - val_accuracy: 0.0077
Epoch 3/5
123603/123603 [==============================] - 172s 1ms/sample - loss: 5.1878 - accuracy: 0.0066 - val_loss: 5.1864 - val_accuracy: 0.0068
Epoch 4/5
123603/123603 [==============================] - 172s 1ms/sample - loss: 5.1876 - accuracy: 0.0061 - val_loss: 5.1865 - val_accuracy: 0.0059
Epoch 5/5
123603/123603 [==============================] - 172s 1ms/sample - loss: 5.1874 - accuracy: 0.0063 - val_loss: 5.1869 - val_accuracy: 0.0067
Train on 123603 samples, validate on 13734 samples
Epoch 6/10
123603/123603 [==============================] - 172s 1ms/sample - loss: 5.1874 - accuracy: 0.0062 - val_loss: 5.1879 - val_accuracy: 0.0077
Epoch 7/10
123603/123603 [==========================

### Load Most Recent Model

In [ ]:
sgd_fine = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
for i in range(len(model.layers)):
    model.layers[i].trainable=False

## Fine-Tuning for Coarse Classifier

In [ ]:
y_train_c = np.dot(y_train,fine2coarse)
y_val_c = np.dot(y_val,fine2coarse)

In [ ]:
net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(1152, activation='elu')(net)
#out_coarse = Dense(20, activation='softmax')(net)
out_coarse = Dense(8, activation='softmax')(net)

model_c = Model(inputs=in_layer,outputs=out_coarse)
model_c.compile(optimizer= sgd_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())

In [ ]:
index = 30
step = 10
stop = 40

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c), callbacks=[tbCallBack])
    index += step

Train on 123603 samples, validate on 13734 samples
Epoch 31/40
123603/123603 [==============================] - 52s 418us/sample - loss: 2.0433 - accuracy: 0.2067 - val_loss: 2.0380 - val_accuracy: 0.2103
Epoch 32/40
123603/123603 [==============================] - 52s 421us/sample - loss: 2.0424 - accuracy: 0.2069 - val_loss: 2.0415 - val_accuracy: 0.2103
Epoch 33/40
123603/123603 [==============================] - 52s 419us/sample - loss: 2.0423 - accuracy: 0.2069 - val_loss: 2.0390 - val_accuracy: 0.2103
Epoch 34/40
123603/123603 [==============================] - 52s 418us/sample - loss: 2.0420 - accuracy: 0.2069 - val_loss: 2.0406 - val_accuracy: 0.2103
Epoch 35/40
123603/123603 [==============================] - 52s 420us/sample - loss: 2.0417 - accuracy: 0.2069 - val_loss: 2.0395 - val_accuracy: 0.2103
Epoch 36/40
123603/123603 [==============================] - 52s 422us/sample - loss: 2.0416 - accuracy: 0.2069 - val_loss: 2.0391 - val_accuracy: 0.2103
Epoch 37/40
123603/123603

In [ ]:
model_c.compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
stop = 50

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c), callbacks=[tbCallBack])
    index += step

Train on 123603 samples, validate on 13734 samples
Epoch 41/50
123603/123603 [==============================] - 52s 423us/sample - loss: 2.0404 - accuracy: 0.2069 - val_loss: 2.0382 - val_accuracy: 0.2103
Epoch 42/50
123603/123603 [==============================] - 52s 421us/sample - loss: 2.0403 - accuracy: 0.2069 - val_loss: 2.0379 - val_accuracy: 0.2103
Epoch 43/50
123603/123603 [==============================] - 52s 418us/sample - loss: 2.0403 - accuracy: 0.2069 - val_loss: 2.0379 - val_accuracy: 0.2103
Epoch 44/50
123603/123603 [==============================] - 52s 419us/sample - loss: 2.0403 - accuracy: 0.2069 - val_loss: 2.0379 - val_accuracy: 0.2103
Epoch 45/50
123603/123603 [==============================] - 52s 419us/sample - loss: 2.0403 - accuracy: 0.2069 - val_loss: 2.0380 - val_accuracy: 0.2103
Epoch 46/50
123603/123603 [==============================] - 52s 419us/sample - loss: 2.0404 - accuracy: 0.2069 - val_loss: 2.0380 - val_accuracy: 0.2103
Epoch 47/50
123603/123603

## Fine-Tuning for Fine Classifiers

### Construct Fine Classifiers

In [ ]:
def fine_model():
    net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
    net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
    net = Dropout(.6)(net)
    net = MaxPooling2D((2, 2), padding='same')(net)

    net = Flatten()(net)
    net = Dense(1152, activation='elu')(net)
#    out_fine = Dense(100, activation='softmax')(net)
    out_fine = Dense(180, activation='softmax')(net)
    model_fine = Model(inputs=in_layer,outputs=out_fine)
    model_fine.compile(optimizer= sgd_coarse,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

In [ ]:
fine_models = {'models' : [{} for i in range(coarse_categories)], 'yhf' : [{} for i in range(coarse_categories)]}
for i in range(coarse_categories):
    model_i = fine_model()
    fine_models['models'][i] = model_i

### Train Fine Classifiers on Respective Data

In [ ]:
def get_error(y,yh):
    # Threshold
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error

In [ ]:
for i in range(coarse_categories):
    index= 0
    step = 5
    stop = 5

    # Get all training data for the coarse category
    ix = np.where([(y_train[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_tix = x_train[ix]
    y_tix = y_train[ix]

    # Get all validation data for the coarse category
    ix_v = np.where([(y_val[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_vix = x_val[ix_v]
    y_vix = y_val[ix_v]

    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step

    fine_models['models'][i].compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
    stop = 10

    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step

    yh_f = fine_models['models'][i].predict(x_val[ix_v], batch_size=batch)
    print('Fine Classifier '+str(i)+' Error: '+str(get_error(y_val[ix_v],yh_f)))

Train on 25579 samples, validate on 2888 samples
Epoch 1/5
25579/25579 [==============================] - 13s 496us/sample - loss: 3.8498 - accuracy: 0.0285 - val_loss: 3.6611 - val_accuracy: 0.0305
Epoch 2/5
25579/25579 [==============================] - 11s 428us/sample - loss: 3.6527 - accuracy: 0.0286 - val_loss: 3.6396 - val_accuracy: 0.0308
Epoch 3/5
25579/25579 [==============================] - 11s 429us/sample - loss: 3.6445 - accuracy: 0.0281 - val_loss: 3.6348 - val_accuracy: 0.0308
Epoch 4/5
25579/25579 [==============================] - 11s 430us/sample - loss: 3.6388 - accuracy: 0.0299 - val_loss: 3.6275 - val_accuracy: 0.0305
Epoch 5/5
25579/25579 [==============================] - 11s 424us/sample - loss: 3.6344 - accuracy: 0.0270 - val_loss: 3.6321 - val_accuracy: 0.0229
Train on 25579 samples, validate on 2888 samples
Epoch 6/10
25579/25579 [==============================] - 12s 454us/sample - loss: 3.6193 - accuracy: 0.0298 - val_loss: 3.6118 - val_accuracy: 0.0301
E

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Fine Classifier 0 Error: 0.9709141274238227
Train on 11911 samples, validate on 1259 samples
Epoch 1/5
11911/11911 [==============================] - 6s 526us/sample - loss: 3.2913 - accuracy: 0.0599 - val_loss: 2.8987 - val_accuracy: 0.0508
Epoch 2/5
11911/11911 [==============================] - 5s 443us/sample - loss: 2.8746 - accuracy: 0.0583 - val_loss: 2.8870 - val_accuracy: 0.0548
Epoch 3/5
11911/11911 [==============================] - 5s 430us/sample - loss: 2.8694 - accuracy: 0.0576 - val_loss: 2.8707 - val_accuracy: 0.0667
Epoch 4/5
11911/11911 [==============================] - 5s 427us/sample - loss: 2.8656 - accuracy: 0.0611 - val_loss: 2.8505 - val_accuracy: 0.0675
Epoch 5/5
11911/11911 [==============================] - 5s 433us/sample - loss: 2.8604 - accuracy: 0.0594 - val_loss: 2.8569 - val_accuracy: 0.0548
Train on 11911 samples, validate on 1259 samples
Epoch 6/10
11911/11911 [==============================] - 6s 485us/sample - loss: 2.8458 - accuracy: 0.0597 - val

## Probabilistic Averaging

In [ ]:
def eval_hdcnn(X, y):
    yh = np.zeros(np.shape(y))

    yh_s = model.predict(X, batch_size=batch)

    print('Single Classifier Error: '+str(get_error(y,yh_s)))

    yh_c = model_c.predict(X, batch_size=batch)
    y_c = np.dot(y,fine2coarse)

    print('Coarse Classifier Error: '+str(get_error(y_c,yh_c)))

    for i in range(coarse_categories):
        if i%5 == 0:
            print("Evaluating Fine Classifier: ", str(i))
        fine_models['yhf'][i] = fine_models['models'][i].predict(X, batch_size=batch)
        yh += np.multiply(yh_c[:,i].reshape((len(y)),1), fine_models['yhf'][i])

    print('Overall Error: '+str(get_error(y,yh)))
    return yh

In [ ]:
yh = eval_hdcnn(x_val,y_val)

Single Classifier Error: 0.9936653560506772
Coarse Classifier Error: 0.7897189456822484
Evaluating Fine Classifier:  0
Evaluating Fine Classifier:  5
Overall Error: 0.9954128440366973
